In [83]:
wm_fw = {'aggregate_1': 0.033624279321068165,
 'aggregate_10': 0.019778035812821653,
 'aggregate_11': 0.015934077647876587,
 'aggregate_12': 0.025189808829097184,
 'aggregate_2': 0.024644849903783968,
 'aggregate_3': 0.031798697555238634,
 'aggregate_4': 0.032011828605370586,
 'aggregate_5': 0.02995528776485348,
 'aggregate_6': 0.056123207049499441,
 'aggregate_7': 0.22862122490843037,
 'aggregate_8': 0.055214974024072216,
 'aggregate_9': 0.073880012751423169,
 'area': 0.028239409774982197,
 'difference_ratio_min_max': 0.018134751267069784,
 'house_num_rooms': 0.022138254919951462,
 'kurtosis': 0.018457866138121292,
 'num_occupants': 0.0089153601644248858,
 'p_25': 0.018201680495129855,
 'p_50': 0.021691809380849931,
 'p_75': 0.054178042595339197,
 'ratio_min_max': 0.016442643589312762,
 'skew': 0.029005353992157302,
 'variance': 0.1378185435091259}

In [192]:
import numpy as np
import pickle
from degree_days import  dd
from regional_average_contribution import  contribution
import pandas as pd
out_overall = pickle.load(open('../data/input/all_regions.pkl','r'))

import sys
import os

train_region, test_region, test_home, appliance, transform, K = "Austin","SanDiego", 5938, "hvac","DD",3
test_home = int(test_home)
K = int(K)

train_df = out_overall[train_region]
test_df = out_overall[test_region]

train_dd = pd.DataFrame(dd[train_region])
test_dd = pd.DataFrame(dd[test_region])

median_aggregate = {}
for region in [train_region, test_region]:
    median_aggregate[region] = {}
    for month in range(1, 13):
        median_aggregate[region][month] = out_overall[region]['aggregate_'+str(month)].median()

median_aggregate_df = pd.DataFrame(median_aggregate)


from itertools import combinations

start_month, end_month = 1,12
agg_features = np.hstack([['aggregate_'+str(month) for month in range(start_month, end_month+1)],
                         'ratio_min_max','difference_ratio_min_max','p_25','p_50','p_75','skew','kurtosis'])
md_features = ['area','house_num_rooms']
features = {'md_agg':np.hstack([
            agg_features,
            md_features
            ]).tolist()}

f_all = features['md_agg']

# Find not null set of common features
def find_com_features_train(df, home_1, home_2, featureset_max):
    f_1 = df.ix[home_1][featureset_max].dropna()
    f_2 = df.ix[home_2][featureset_max].dropna()
    com_f =  np.intersect1d(f_1.index, f_2.index)
    return com_f


def find_distance_train_test_quick(df_train, home_1, home_2, df_test, home_test, featureset_train, featureset_max):
    f_test = df_test.ix[home_test][featureset_max].dropna()
    com_f =  np.intersect1d(f_test.index, featureset_train)
    if len(com_f):
        is_common = True
    else:
        is_common = False
        return is_common, None
    
    

    if len(com_f):
        a = (df_train.ix[home_1][com_f]- df_test.ix[home_test][com_f]).abs().sum().sum()
        b = (df_train.ix[home_2][com_f]- df_test.ix[home_test][com_f]).abs().sum().sum()
        
        if a<=b:
            order = [home_1, home_2]
        else:
            order = [home_2, home_1]
        return is_common, {'order':order,
                    'num_f':len(com_f),
                    'dist_a':a,
                    'dist_b':b,
                          'f':com_f}


def find_distance_train_test_dot(df_train, home_1, home_2, df_test, home_test, featureset_train, featureset_max):
    f_test = df_test.ix[home_test][featureset_max].dropna()
    com_f =  np.intersect1d(f_test.index, featureset_train)
    if len(com_f):
        is_common = True
    else:
        is_common = False
        return is_common, None

    if len(com_f):
        fw = pd.Series({f:wm_fw[f] for f in com_f})
        a = np.linalg.norm((df_train.ix[home_1][com_f]- df_test.ix[home_test][com_f]).dot(fw))
        b = np.linalg.norm((df_train.ix[home_2][com_f]- df_test.ix[home_test][com_f]).dot(fw))
        if a<=b:
            order = [home_1, home_2]
        else:
            order = [home_2, home_1]
        return is_common, {'order':order,
                    'num_f':len(com_f),
                    'dist_a':a,
                    'dist_b':b,
                          'f':com_f}
    
def find_distance_train_test(df_train, home_1, home_2, df_test, home_test, featureset_train, featureset_max):
    f_test = df_test.ix[home_test][featureset_max].dropna()
    com_f =  np.intersect1d(f_test.index, featureset_train)
    if len(com_f):
        is_common = True
    else:
        is_common = False
        return is_common, None

    if len(com_f):
        fw = pd.Series({f:wm_fw[f] for f in com_f})
        a = np.linalg.norm((df_train.ix[home_1][com_f]- df_test.ix[home_test][com_f]))
        b = np.linalg.norm((df_train.ix[home_2][com_f]- df_test.ix[home_test][com_f]))
        if a<=b:
            order = [home_1, home_2]
        else:
            order = [home_2, home_1]
        return is_common, {'order':order,
                    'num_f':len(com_f),
                    'dist_a':a,
                    'dist_b':b,
                          'f':com_f}

import pandas as pd

def scale_0_1(ser, minimum=None, maximum=None):
    if minimum is not None:
        pass
    else:
        minimum = ser.min()
        maximum = ser.max()
    return (ser-minimum).div(maximum-minimum)

def normalise(df):
    new_df = df.copy()
    max_aggregate = df[["aggregate_%d" % i for i in range(1, 13)]].max().max()
    min_aggregate = df[["aggregate_%d" % i for i in range(1, 13)]].min().min()
    new_df[["aggregate_%d" % i for i in range(1, 13)]] = scale_0_1(df[["aggregate_%d" % i for i in range(1, 13)]], min_aggregate, max_aggregate)
    for col in ['area','num_occupants','house_num_rooms','ratio_min_max',
                'skew','kurtosis','variance','difference_ratio_min_max','p_25',
               'p_50','p_75']:
        new_df[col] = scale_0_1(df[col])
    return new_df

if transform in ["None","None-percentage"]:
    pass
elif transform in ["DD","DD-percentage"]:
    train_df_copy = train_df.copy()
    for month in range(1, 13):
        # index on 0, 11
        train_dd_month = train_dd.ix[month-1]['Total']
        test_dd_month = test_dd.ix[month-1]['Total']
        train_df['hvac_%d' % month] = train_df_copy['hvac_%d' % month] * test_dd_month*1. / train_dd_month

        #New aggregate will be removing old HVAC and adding new HVAC!
        train_df['aggregate_%d' %month] = train_df_copy['aggregate_%d' %month] - train_df_copy['hvac_%d' % month] + train_df['hvac_%d' % month]
elif transform in ["median-aggregate","median-aggregate-percentage"]:
    train_df_copy = train_df.copy()
    for month in range(1,13):
        median_month = median_aggregate_df.ix[month]
        cols_to_transform = [x for x in train_df.columns if "_"+str(month) in x]
        train_df[cols_to_transform] = train_df_copy[cols_to_transform] * median_month[test_region] / median_month[train_region]

elif transform in ["regional","regional-percentage"]:
    train_df_copy = train_df.copy()
    for month in range(1, 13):

        # index on 0, 11
        if month in range(4,11):
            mode='Cooling'
        else:
            mode='Heating'

        train_dd_month = contribution[train_region][mode]['hvac']
        test_dd_month = contribution[test_region][mode]['hvac']

        train_df['hvac_%d' % month] = train_df_copy['hvac_%d' % month] * test_dd_month*1. / train_dd_month

        #New aggregate will be removing old HVAC and adding new HVAC!
        train_df['aggregate_%d' %month] = train_df_copy['aggregate_%d' %month] - train_df_copy['hvac_%d' % month] + train_df['hvac_%d' % month]



elif transform=="DD-fridge":
    train_df_copy = train_df.copy()
    fridge_model = pickle.load(open('../data/input/SanDiego_fridge_dd_coef.pkl','r'))
    for month in range(1, 13):
        # index on 0, 11

        train_cdd_month = train_dd.ix[month-1]['Cooling']
        test_cdd_month = test_dd.ix[month-1]['Cooling']
        for fridge_home, fridge_home_model in fridge_model.iteritems():
            train_df.loc[fridge_home,'fridge_%d' % month] = fridge_home_model['baseline']+fridge_home_model['cdd']*test_cdd_month
            train_df.loc[fridge_home,'aggregate_%d' %month] = train_df_copy.ix[fridge_home]['aggregate_%d' %month] - train_df_copy.ix[fridge_home]['fridge_%d' % month] + train_df.ix[fridge_home]['fridge_%d' % month]

overall_df = pd.concat([train_df, test_df])

normalised_df = normalise(overall_df)

train_normalised_df = normalised_df.ix[train_df.index]
test_normalised_df = normalised_df.ix[test_df.index].drop_duplicates()


def solve_ilp(inequalities, time_limit=50):
    from collections import defaultdict
    import pandas as pd
    co = defaultdict(int)
    for ineq in inequalities:
        lt = ineq[0]
        gt = ineq[1]
        co[lt]-= 1
        co[gt]+= 1
    co_ser = pd.Series(co)
    co_ser.sort()

    return co_ser.index.values.tolist()

In [193]:
month_compute=1
candidate_homes = train_normalised_df['%s_%d' %(appliance, month_compute)].dropna().index.values

In [194]:
len(candidate_homes)

417

In [195]:
candidate_homes

array([  22,   26,   59,   77,   86,   93,   94,  101,  114,  115,  121,
        130,  135,  160,  187,  222,  267,  297,  347,  364,  370,  410,
        434,  436,  457,  484,  491,  499,  503,  508,  545,  580,  624,
        645,  661,  668,  739,  744,  772,  781,  796,  871,  878,  898,
        936,  946,  954,  974,  980,  994, 1037, 1069, 1086, 1105, 1169,
       1185, 1192, 1202, 1283, 1310, 1314, 1331, 1334, 1403, 1415, 1463,
       1479, 1500, 1507, 1551, 1586, 1589, 1617, 1632, 1642, 1681, 1697,
       1700, 1714, 1718, 1790, 1791, 1796, 1800, 1854, 1889, 1947, 1953,
       1994, 2004, 2018, 2034, 2094, 2129, 2156, 2158, 2171, 2199, 2233,
       2242, 2335, 2361, 2365, 2366, 2378, 2401, 2458, 2461, 2470, 2472,
       2510, 2520, 2532, 2557, 2575, 2638, 2641, 2751, 2755, 2769, 2787,
       2814, 2818, 2829, 2845, 2859, 2864, 2907, 2945, 2953, 2965, 2974,
       2992, 3009, 3036, 3039, 3044, 3134, 3192, 3268, 3273, 3299, 3310,
       3367, 3411, 3413, 3425, 3443, 3456, 3482, 34

In [196]:
candidate_homes = np.array(np.setdiff1d(candidate_homes, test_home))

In [197]:
len(candidate_homes)

417

In [198]:
a,b= next(combinations(candidate_homes, 2))
a,b
com_features = find_com_features_train(train_df, a, b, f_all)

In [199]:
com_features

array(['aggregate_1', 'aggregate_2', 'aggregate_3', 'aggregate_4',
       'aggregate_5', 'aggregate_6'], dtype=object)

In [200]:
preds = {}
gts = {}
for month_compute in range(1, 13):

    from collections import Counter, defaultdict
    num_features_all = {}
    ineq_dict = {}

    num_features_all[appliance] = {}
    ineq_dict[appliance] = {}

    #num_features_all[appliance][month_compute] = {}
    ineq_dict[appliance][month_compute] = {}

    candidate_homes = train_normalised_df['%s_%d' %(appliance, month_compute)].dropna().index.values[:30]



    #num_features_all[appliance][month_compute][test_home] = defaultdict(int)
    from collections import defaultdict
    import pandas as pd
    co = defaultdict(int)
    store_path = '../../../output/output/ineq_cross/%s_%s_%s_%s_%d_%d_%d.pkl' %(train_region,
                                                                        test_region,
                                                                        transform,
                                                                        appliance,
                                                                        month_compute,
                                                                        test_home, K)
    print store_path
    if os.path.exists(store_path):
        print "already exists"
        


    if not np.isnan(test_normalised_df.ix[test_home]['%s_%d' %(appliance, month_compute)]):
        # We need to predict this value!
        # Find candidate set, train homes which have not null for this month
        # Now find features on pairs of homes in candidate homes
        for a,b in combinations(candidate_homes, 2):
            com_features = find_com_features_train(train_df, a, b, f_all)
            

            if len(com_features):
                # Consider a,b
                is_common, d = find_distance_train_test_dot(train_normalised_df, a, b, test_normalised_df, test_home, com_features, f_all)
                if is_common:

                    # Common between train and test. Can add this pair to inequalities
                    ineq=d['order']
                    lt = ineq[0]
                    gt = ineq[1]
                    co[lt]-= 1
                    co[gt]+= 1
                    #num_features_all[appliance][month_compute][test_home][d['num_f']]+= 1

        """
        # Saving ineqs
        pickle.dump(ineqs, open('../data/model/inequalities/%s_%s_%s_%s_%d_%d.pkl' %(train_region,
                                                                        test_region,
                                                                        transform,
                                                                        appliance,
                                                                        month_compute,
                                                                        test_home),'w'))
        """
        co_ser = pd.Series(co)
        co_ser.sort()
        ranks = co_ser.index.values.tolist()
        if "percentage" in transform:
            mean_proportion = (train_df.ix[ranks[:K]]['%s_%d' %(appliance, month_compute)]/ train_df.ix[ranks[:K]]['aggregate_%d' %(month_compute)]).mean()

            pred = test_df.ix[test_home]['aggregate_%d' %month_compute]*mean_proportion

        else:
            pred = train_df.ix[ranks[:K]]['%s_%d' %(appliance, month_compute)].dropna().mean()
        gt = test_df.ix[test_home]['%s_%d' %(appliance, month_compute)]
        
        #pickle.dump(pred, open(store_path,'w'))
        preds[month_compute]=pred
        gts[month_compute] = gt

    else:
        # No need to predict
        pass

../../../output/output/ineq_cross/Austin_SanDiego_DD_hvac_1_5938_3.pkl
../../../output/output/ineq_cross/Austin_SanDiego_DD_hvac_2_5938_3.pkl
../../../output/output/ineq_cross/Austin_SanDiego_DD_hvac_3_5938_3.pkl
../../../output/output/ineq_cross/Austin_SanDiego_DD_hvac_4_5938_3.pkl
../../../output/output/ineq_cross/Austin_SanDiego_DD_hvac_5_5938_3.pkl
../../../output/output/ineq_cross/Austin_SanDiego_DD_hvac_6_5938_3.pkl
../../../output/output/ineq_cross/Austin_SanDiego_DD_hvac_7_5938_3.pkl
../../../output/output/ineq_cross/Austin_SanDiego_DD_hvac_8_5938_3.pkl
../../../output/output/ineq_cross/Austin_SanDiego_DD_hvac_9_5938_3.pkl
../../../output/output/ineq_cross/Austin_SanDiego_DD_hvac_10_5938_3.pkl
../../../output/output/ineq_cross/Austin_SanDiego_DD_hvac_11_5938_3.pkl
../../../output/output/ineq_cross/Austin_SanDiego_DD_hvac_12_5938_3.pkl


/Users/nipunbatra/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:66: FutureWarning: sort is deprecated, use sort_values(inplace=True) for for INPLACE sorting


In [201]:
pred_gt_df = pd.DataFrame({'pred':preds,"gt":gts})
error = (pred_gt_df['pred']-pred_gt_df['gt']).abs().div(pred_gt_df['gt'])
error = error*100
acc = 100-error
acc[acc<0]= 0
acc.ix[5:10].mean()


42.21648935239287

In [202]:
pred_gt_df

,gt,pred
1,0.016150,24.530744
2,0.023167,29.559608
3,0.020900,61.015241
4,0.021433,35.091023
5,0.021450,27.772422
6,0.021017,61.284750
7,52.150852,78.587438
8,120.904800,94.008903
9,204.430679,105.302012
10,50.629719,63.425320


In [203]:
train_df.ix[ranks[:K]]

,hvac_1,hvac_2,hvac_3,hvac_4,hvac_5,hvac_6,hvac_7,hvac_8,hvac_9,hvac_10,...,md_available,full_agg_available,ratio_min_max,difference_ratio_min_max,variance,skew,kurtosis,p_25,p_50,p_75
94,0.000000,0.000099,0.000091,3.650819,6.204708,31.297296,68.286877,99.212429,78.964664,40.053538,...,1,1,0.383497,0.383497,11482.221828,0.894630,0.234951,294.383591,347.958496,388.923569
160,0.000099,0.000222,0.000045,3.276201,17.883578,120.029017,205.059758,288.667527,300.507095,128.225382,...,0,1,0.264077,0.264077,36483.275792,0.244571,-1.309636,273.358307,425.754730,558.868637
101,0.035663,0.338140,13.533932,63.568737,60.043989,153.080112,250.527483,310.467082,297.662111,230.978650,...,1,1,0.303611,0.303611,47735.228222,0.529240,-1.204214,325.532219,476.204910,700.718491


In [204]:
preds = {}
gts = {}
for month_compute in range(1, 13):

    from collections import Counter, defaultdict
    num_features_all = {}
    ineq_dict = {}

    num_features_all[appliance] = {}
    ineq_dict[appliance] = {}

    #num_features_all[appliance][month_compute] = {}
    ineq_dict[appliance][month_compute] = {}

    candidate_homes = train_normalised_df['%s_%d' %(appliance, month_compute)].dropna().index.values[:30]



    #num_features_all[appliance][month_compute][test_home] = defaultdict(int)
    from collections import defaultdict
    import pandas as pd
    co = defaultdict(int)
    store_path = '../../../output/output/ineq_cross/%s_%s_%s_%s_%d_%d_%d.pkl' %(train_region,
                                                                        test_region,
                                                                        transform,
                                                                        appliance,
                                                                        month_compute,
                                                                        test_home, K)
    print store_path
    if os.path.exists(store_path):
        print "already exists"
        


    if not np.isnan(test_normalised_df.ix[test_home]['%s_%d' %(appliance, month_compute)]):
        # We need to predict this value!
        # Find candidate set, train homes which have not null for this month
        # Now find features on pairs of homes in candidate homes
        for a,b in combinations(candidate_homes, 2):
            com_features = find_com_features_train(train_df, a, b, f_all)

            if len(com_features):
                # Consider a,b
                is_common, d = find_distance_train_test(train_normalised_df, a, b, test_normalised_df, test_home, com_features, f_all)
                if is_common:

                    # Common between train and test. Can add this pair to inequalities
                    ineq=d['order']
                    lt = ineq[0]
                    gt = ineq[1]
                    co[lt]-= 1
                    co[gt]+= 1
                    #num_features_all[appliance][month_compute][test_home][d['num_f']]+= 1

        """
        # Saving ineqs
        pickle.dump(ineqs, open('../data/model/inequalities/%s_%s_%s_%s_%d_%d.pkl' %(train_region,
                                                                        test_region,
                                                                        transform,
                                                                        appliance,
                                                                        month_compute,
                                                                        test_home),'w'))
        """
        co_ser = pd.Series(co)
        co_ser.sort()
        ranks = co_ser.index.values.tolist()
        if "percentage" in transform:
            mean_proportion = (train_df.ix[ranks[:K]]['%s_%d' %(appliance, month_compute)]/ train_df.ix[ranks[:K]]['aggregate_%d' %(month_compute)]).mean()

            pred = test_df.ix[test_home]['aggregate_%d' %month_compute]*mean_proportion

        else:
            pred = train_df.ix[ranks[:K]]['%s_%d' %(appliance, month_compute)].dropna().mean()
        gt = test_df.ix[test_home]['%s_%d' %(appliance, month_compute)]
        preds[month_compute]=pred
        gts[month_compute] = gt


    else:
        # No need to predict
        pass

../../../output/output/ineq_cross/Austin_SanDiego_DD_hvac_1_5938_3.pkl
../../../output/output/ineq_cross/Austin_SanDiego_DD_hvac_2_5938_3.pkl
../../../output/output/ineq_cross/Austin_SanDiego_DD_hvac_3_5938_3.pkl
../../../output/output/ineq_cross/Austin_SanDiego_DD_hvac_4_5938_3.pkl
../../../output/output/ineq_cross/Austin_SanDiego_DD_hvac_5_5938_3.pkl
../../../output/output/ineq_cross/Austin_SanDiego_DD_hvac_6_5938_3.pkl
../../../output/output/ineq_cross/Austin_SanDiego_DD_hvac_7_5938_3.pkl
../../../output/output/ineq_cross/Austin_SanDiego_DD_hvac_8_5938_3.pkl
../../../output/output/ineq_cross/Austin_SanDiego_DD_hvac_9_5938_3.pkl
../../../output/output/ineq_cross/Austin_SanDiego_DD_hvac_10_5938_3.pkl
../../../output/output/ineq_cross/Austin_SanDiego_DD_hvac_11_5938_3.pkl
../../../output/output/ineq_cross/Austin_SanDiego_DD_hvac_12_5938_3.pkl


/Users/nipunbatra/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:65: FutureWarning: sort is deprecated, use sort_values(inplace=True) for for INPLACE sorting


In [205]:
pred_gt_df = pd.DataFrame({'pred':preds,"gt":gts})
error = (pred_gt_df['pred']-pred_gt_df['gt']).abs().div(pred_gt_df['gt'])
error = error*100
acc = 100-error
acc[acc<0]= 0
acc
acc.ix[5:10].mean()

36.97621331345112

In [191]:
pred_gt_df

,gt,pred
1,0.000000,2.247812
2,0.000000,2.620437
3,22.754534,17.103252
4,0.971283,52.333696
5,3.498750,54.323677
6,180.404144,163.182191
7,392.775696,353.390685
8,537.882385,468.226187
9,342.975098,417.291159
10,131.348648,258.197011


In [162]:
train_df.ix[ranks[:K]]

,hvac_1,hvac_2,hvac_3,hvac_4,hvac_5,hvac_6,hvac_7,hvac_8,hvac_9,hvac_10,...,md_available,full_agg_available,ratio_min_max,difference_ratio_min_max,variance,skew,kurtosis,p_25,p_50,p_75
781,0.227985,2.661724,33.975815,109.211413,95.898007,236.582860,332.564956,498.708526,456.471371,362.444898,...,0,1,0.331930,0.331930,87381.897888,0.618144,-0.966644,538.501038,707.339874,1045.020660
624,60.013319,37.594527,101.620227,0.000094,22.751338,89.836481,166.750600,249.301381,203.469019,26.507666,...,1,1,0.416310,0.416310,26668.198555,0.517960,-1.135187,454.631447,513.213882,693.046631
77,185.334888,178.997564,347.181591,139.976891,99.224158,279.011967,416.630039,546.533608,494.094518,348.255390,...,0,1,0.477851,0.477851,74286.882561,0.135571,-1.122693,960.258499,1196.487000,1390.395142


In [141]:
test_df.ix[test_home]

hvac_1                          0.002000
hvac_2                          0.002700
hvac_3                         29.053850
hvac_4                         19.651184
hvac_5                         15.861900
hvac_6                         68.252647
hvac_7                        145.872055
hvac_8                        173.272644
hvac_9                        203.351685
hvac_10                        72.101601
hvac_11                         0.619017
hvac_12                         2.839517
wm_1                                 NaN
wm_2                                 NaN
wm_3                                 NaN
wm_4                                 NaN
wm_5                                 NaN
wm_6                                 NaN
wm_7                                 NaN
wm_8                                 NaN
wm_9                                 NaN
wm_10                                NaN
wm_11                                NaN
wm_12                                NaN
fridge_1        

In [118]:
is_common, d = find_distance_train_test_dot(train_normalised_df, a, b, test_normalised_df, test_home, com_features, f_all)

In [119]:
d

{'dist_a': 0.0059220825948114141,
 'dist_b': 0.0036127385696059933,
 'f': array(['aggregate_1', 'aggregate_10', 'aggregate_11', 'aggregate_12',
        'aggregate_2', 'aggregate_3', 'aggregate_4', 'aggregate_5',
        'aggregate_6', 'aggregate_7', 'aggregate_8', 'aggregate_9',
        'difference_ratio_min_max', 'kurtosis', 'p_25', 'p_50', 'p_75',
        'ratio_min_max', 'skew'], dtype=object),
 'num_f': 19,
 'order': [160, 115]}

In [120]:
is_common, d = find_distance_train_test(train_normalised_df, a, b, test_normalised_df, test_home, com_features, f_all)

In [121]:
d

{'dist_a': 0.16441051522414468,
 'dist_b': 0.29400373367791882,
 'f': array(['aggregate_1', 'aggregate_10', 'aggregate_11', 'aggregate_12',
        'aggregate_2', 'aggregate_3', 'aggregate_4', 'aggregate_5',
        'aggregate_6', 'aggregate_7', 'aggregate_8', 'aggregate_9',
        'difference_ratio_min_max', 'kurtosis', 'p_25', 'p_50', 'p_75',
        'ratio_min_max', 'skew'], dtype=object),
 'num_f': 19,
 'order': [115, 160]}

In [136]:
train_df.ix[ranks[:K]]

,hvac_1,hvac_2,hvac_3,hvac_4,hvac_5,hvac_6,hvac_7,hvac_8,hvac_9,hvac_10,...,md_available,full_agg_available,ratio_min_max,difference_ratio_min_max,variance,skew,kurtosis,p_25,p_50,p_75
94,0.000000,0.000099,0.000091,3.650819,6.204708,31.297296,68.286877,99.212429,78.964664,40.053538,...,1,1,0.383497,0.383497,11482.221828,0.894630,0.234951,294.383591,347.958496,388.923569
115,0.000000,0.000491,3.299546,25.147722,25.659188,83.801032,150.657918,179.216002,158.120355,44.306730,...,0,1,0.356844,0.356844,13482.923919,0.396173,-1.150852,243.321659,297.264175,413.592415
101,0.035663,0.338140,13.533932,63.568737,60.043989,153.080112,250.527483,310.467082,297.662111,230.978650,...,1,1,0.303611,0.303611,47735.228222,0.529240,-1.204214,325.532219,476.204910,700.718491


In [142]:
test_df

,hvac_1,hvac_2,hvac_3,hvac_4,hvac_5,hvac_6,hvac_7,hvac_8,hvac_9,hvac_10,...,md_available,full_agg_available,ratio_min_max,difference_ratio_min_max,variance,skew,kurtosis,p_25,p_50,p_75
54,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
203,0.002000,0.002700,29.053850,19.651184,15.861900,68.252647,145.872055,173.272644,203.351685,72.101601,...,0,1,0.352858,0.352858,11444.769891,0.990983,-0.052504,241.198689,268.654037,351.088257
527,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1450,1.563983,49.754932,224.037476,105.825180,54.593281,266.363251,365.305908,497.618652,548.815247,326.567413,...,0,1,0.414237,0.414237,56334.856003,0.276487,-1.620418,601.410309,726.297211,979.907196
1524,0.025400,0.027333,16.035267,4.751067,14.896517,185.289536,218.260483,325.225800,331.544067,159.814423,...,0,1,0.323581,0.323581,29697.892286,0.737280,-0.923025,257.036800,330.575531,507.019714
1731,0.197017,0.000000,21.463934,12.613634,7.330667,40.226986,221.842163,286.762146,295.495148,175.822769,...,1,1,0.360746,0.360746,39154.448822,0.087290,-2.083153,329.804420,490.044968,692.856308
2031,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2354,0.000517,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2606,4.041283,48.951168,114.115364,32.597801,30.465033,118.778206,213.009796,302.353729,358.428986,233.708633,...,1,1,0.440386,0.440386,29220.154500,0.648063,-0.327643,536.360283,607.543243,779.882858
3687,0.000000,0.000000,29.437967,0.000000,0.000000,69.320015,183.210800,273.997559,13.414872,NaN,...,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
